In [1]:
import torch
import numpy as np

c:\Users\ekzms\.conda\envs\cooling\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
estimator = TimeGradEstimator(
    input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
    hidden_size=64,
    num_layers=2,
    dropout_rate=0.1,
    # target_dim = int(dataset.metadata.feat_static_cat[0].cardinality),
    lags_seq=[1],
    scheduler=scheduler,
    num_inference_steps=10,
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    scaling="mean",
    trainer_kwargs=dict(max_epochs=50, accelerator="gpu", devices="auto"),
    )

In [ ]:
model_kwargs={
                "freq": self.freq,
                "context_length": self.context_length,
                "prediction_length": self.prediction_length,
                "scheduler": self.scheduler,
                "num_inference_steps": self.num_inference_steps,
                "input_size": self.input_size,
                "num_feat_dynamic_real": (
                    1 + self.num_feat_dynamic_real + len(self.time_features)
                ),
                "num_feat_static_real": max(1, self.num_feat_static_real),
                "num_feat_static_cat": max(1, self.num_feat_static_cat),
                "cardinality": self.cardinality,
                "embedding_dimension": self.embedding_dimension,
                "num_layers": self.num_layers,
                "hidden_size": self.hidden_size,
                "dropout_rate": self.dropout_rate,
                "lags_seq": self.lags_seq,
                "scaling": self.scaling,
                "default_scale": self.default_scale,
                "num_parallel_samples": self.num_parallel_samples,
            },
        )

In [2]:
from typing import List, Optional, Tuple, Union
from pts.util import lagged_sequence_values
from gluonts.torch.util import repeat_along_dim, unsqueeze_expand
from gluonts.torch.modules.feature import FeatureEmbedder
from gluonts.torch.scaler import MeanScaler, NOPScaler, Scaler, StdScaler

def prepare_rnn_input(
        feat_static_cat: torch.Tensor,
        feat_static_real: torch.Tensor,
        past_time_feat: torch.Tensor,
        past_target: torch.Tensor,
        past_observed_values: torch.Tensor,
        future_time_feat: torch.Tensor,
        input_size : int,
        future_target: Optional[torch.Tensor] = None,
        cardinality: List[int] = [1],
        embedding_dimension : int = 5,
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor,]:
        context = past_target[:, -12 :, ...]
        observed_context = past_observed_values[:, -12 :, ...]
        
        scaler: Scaler = MeanScaler(
                dim=1, keepdim=True, default_scale=0.0
            )
        
        embedder = FeatureEmbedder(
            cardinalities=cardinality,
            embedding_dims=embedding_dimension,
        )
        input, loc, scale = scaler(context, observed_context)
        future_length = future_time_feat.shape[-2]
        if future_length > 1:
            assert future_target is not None
            input = torch.cat(
                (input, (future_target[:, : future_length - 1, ...] - loc) / scale),
                dim=1,
            )
        prior_input = (past_target[:, : -12, ...] - loc) / scale

        lags = lagged_sequence_values([1], prior_input, input, dim=1)
        time_feat = torch.cat(
            (
                past_time_feat[:, -12 + 1 :, ...],
                future_time_feat,
            ),
            dim=1,
        )

        embedded_cat = embedder(feat_static_cat)
        log_abs_loc = (
            loc.abs().log1p() if input_size == 1 else loc.squeeze(1).abs().log1p()
        )
        log_scale = scale.log() if input_size == 1 else scale.squeeze(1).log()

        static_feat = torch.cat(
            (embedded_cat, feat_static_real, log_abs_loc, log_scale),
            dim=-1,
        )
        expanded_static_feat = unsqueeze_expand(
            static_feat, dim=1, size=time_feat.shape[-2]
        )

        features = torch.cat((expanded_static_feat, time_feat), dim=-1)

        return torch.cat((lags, features), dim=-1), loc, scale, static_feat

c:\Users\ekzms\.conda\envs\cooling\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')
c:\Users\ekzms\.conda\envs\cooling\lib\site-packages\gluonts\json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
c:\Users\ekzms\.conda\envs\cooling\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
prepare_rnn_input_ex = prepare_rnn_input()

In [3]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

In [4]:
from pts.model.tempflow import TempFlowEstimator
from pts.model.time_grad import TimeGradEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer

In [5]:
# exchange_rate_nips, electricity_nips, traffic_nips, solar_nips, wiki-rolling_nips, ## taxi_30min is buggy still
dataset = get_dataset("electricity_nips", regenerate=False)

In [6]:
train_grouper = MultivariateGrouper(max_target_dim=min(2000, int(dataset.metadata.feat_static_cat[0].cardinality)))
dataset_train = train_grouper(dataset.train)

In [7]:
dataset_train

[{'target': array([[115.27613 , 115.27613 , 115.27613 , ...,  71.42857 ,  52.99539 ,
           52.419353],
         [175.53192 , 164.89362 , 154.78723 , ..., 128.7234  , 129.78723 ,
          116.489365],
         [ 31.993204,  31.28539 ,  31.568516, ...,  63.703285,  45.58324 ,
           41.19479 ],
         ...,
         [ 90.3149  ,  93.01075 ,  90.31874 , ..., 139.51228 , 111.4593  ,
          114.91551 ],
         [114.57534 , 120.746574, 126.23288 , ..., 238.14383 , 137.887   ,
          132.05821 ],
         [ 47.912006,  46.234154,  47.53915 , ...,  90.23117 ,  66.74124 ,
           59.470543]], dtype=float32),
  'start': Period('2014-01-01 00:00', 'H'),
  'feat_static_cat': array([0])}]

In [8]:
from pts.model.time_grad.lightning_module import TimeGradLightningModule

In [10]:
from diffusers import DEISMultistepScheduler
scheduler = DEISMultistepScheduler( num_train_timesteps=150, beta_end=0.1, )

In [ ]:
self.freq = freq
self.prediction_length = prediction_length
self.input_size = input_size
self.scheduler = scheduler
self.num_inference_steps = num_inference_steps
self.context_length = (
    context_length if context_length is not None else prediction_length
)
self.patience = patience

self.num_layers = num_layers
self.hidden_size = hidden_size
self.lr = 1e-3
self.weight_decay = weight_decay
self.dropout_rate = dropout_rate
self.num_feat_dynamic_real = num_feat_dynamic_real
self.num_feat_static_cat = num_feat_static_cat
self.num_feat_static_real = num_feat_static_real
self.cardinality = (
    cardinality if cardinality and num_feat_static_cat > 0 else [1]
)
self.embedding_dimension = embedding_dimension
self.scaling = scaling
self.default_scale = default_scale
self.lags_seq = lags_seq
self.time_features = (
    time_features
    if time_features is not None
    else time_features_from_frequency_str(self.freq)
)

self.num_parallel_samples = num_parallel_samples
self.batch_size = batch_size
self.num_batches_per_epoch = num_batches_per_epoch

In [9]:
input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
hidden_size=64,
num_layers=2,
dropout_rate=0.1,
# target_dim = int(dataset.metadata.feat_static_cat[0].cardinality),
lags_seq=[1],
#scheduler=scheduler,
num_inference_steps=10,
prediction_length=dataset.metadata.prediction_length,
context_length=dataset.metadata.prediction_length,
freq=dataset.metadata.freq,
scaling="mean",
trainer_kwargs=dict(max_epochs=50, accelerator="gpu", devices="auto"),


In [15]:
from gluonts.time_feature import TimeFeature, time_features_from_frequency_str
time_features = None
time_features = (
            time_features
            if time_features is not None
            else time_features_from_frequency_str('H')
        )

In [19]:
cardinality = None
num_feat_static_cat = 0
cardinality = (
            cardinality if cardinality and num_feat_static_cat > 0 else [1]
        )

In [21]:
TimeGradLightningModule(
            # loss=loss,
            lr=1e-3,
            weight_decay=1e-8,
            patience=10,
            model_kwargs={
                "freq": freq,
                "context_length": context_length,
                "prediction_length": prediction_length,
                "scheduler": scheduler,
                "num_inference_steps": num_inference_steps,
                "input_size": input_size,
                "num_feat_dynamic_real": (
                    1 + 0 + len(time_features)
                ),
                "num_feat_static_real": max(1, 0),
                "num_feat_static_cat": max(1, 0),
                "cardinality": cardinality,
                "embedding_dimension": None,
                "num_layers": num_layers,
                "hidden_size": hidden_size,
                "dropout_rate": dropout_rate,
                "lags_seq": lags_seq,
                "scaling": scaling,
                "default_scale": 0.0,
                "num_parallel_samples": 100,
            },
        )

ValidationError: 10 validation errors for TimeGradModelModel
freq
  str type expected (type=type_error.str)
context_length
  value is not a valid integer (type=type_error.integer)
prediction_length
  value is not a valid integer (type=type_error.integer)
input_size
  value is not a valid integer (type=type_error.integer)
num_layers
  value is not a valid integer (type=type_error.integer)
hidden_size
  value is not a valid integer (type=type_error.integer)
dropout_rate
  value is not a valid float (type=type_error.float)
lags_seq -> 0
  value is not a valid integer (type=type_error.integer)
scaling
  str type expected (type=type_error.str)
num_inference_steps
  value is not a valid integer (type=type_error.integer)

In [ ]:
input_size: torch.Size([1, 24, 370])
input_size: torch.Size([1, 24, 370])
prior_input_size: torch.Size([1, 0, 370])
lags_size: torch.Size([1, 24, 370])
time_feat_size: torch.Size([1, 24, 5])
static_feat_size: torch.Size([1, 742])
expanded_static_feat_size: torch.Size([1, 24, 742])
features_size: torch.Size([1, 24, 747])
torch.Size([1, 24, 1117])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')
------
torch.Size([1, 24, 64])

In [22]:
aaa = torch.rand((140,20))

In [23]:
aaa

tensor([[0.4495, 0.1000, 0.3259,  ..., 0.0753, 0.8908, 0.6085],
        [0.2675, 0.3706, 0.5410,  ..., 0.6440, 0.7569, 0.1065],
        [0.0845, 0.1099, 0.7057,  ..., 0.6930, 0.2216, 0.4199],
        ...,
        [0.7294, 0.7986, 0.0877,  ..., 0.9380, 0.1775, 0.2543],
        [0.0922, 0.4064, 0.9166,  ..., 0.8525, 0.5120, 0.1626],
        [0.5605, 0.3524, 0.6727,  ..., 0.9560, 0.4364, 0.9747]])

In [25]:
A = torch.eye(20)

In [27]:
A.size()

torch.Size([20, 20])

In [30]:
supports = [A,A]
weights = torch.nn.Parameter(torch.FloatTensor(2*3*20, 20))

In [36]:
aaa = torch.rand((24,20))
A = torch.eye(20)
supports = [A,A]
weights = torch.nn.Parameter(torch.FloatTensor(2*3*20, 20))
x_g = []        
support_set = []
for support in supports:
    support_ks = [torch.eye(support.shape[0]).to(support.device), support]
    for k in range(2, 3):
        support_ks.append(torch.matmul(2 * support, support_ks[-1]) - support_ks[-2]) 
    support_set.extend(support_ks)
for support in support_set:
    x_g.append(torch.einsum("nm,bmc->bnc", support, aaa))
x_g = torch.cat(x_g, dim=-1) # B, N, 2 * cheb_k * dim_in
x_gconv = torch.einsum('bni,io->bno', x_g, weights)  # b, N, dim_out

RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (2) for operand 1 and no ellipsis was given

In [59]:
aaa = torch.rand((24,20))
A = torch.rand(20, 20)
B = torch.rand(20, 20)
supports = [A, B]
weights = torch.nn.Parameter(torch.FloatTensor(2 * 3 * 20, 20))
support_set = []
for support in supports:
    support_ks = [torch.eye(support.shape[0]).to(support.device), support]
    for k in range(2, 3):
        support_ks.append(torch.matmul(2 * support, support_ks[-1]) - support_ks[-2]) 
    support_set.extend(support_ks)

#x_g = torch.cat(x_g, dim=-1)  # B, N, 2 * cheb_k * dim_in

# Perform matrix multiplication using torch.matmul
x_gconv = torch.matmul(x_g, weights)  # b, N, dim_out


In [60]:
x_g = []  
for support in support_set:
    print(support.size())
    x_g.append(torch.matmul(aaa, support))

torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([20, 20])
torch.Size([20, 20])


In [64]:
x_g = torch.cat(x_g, dim=-1) 

In [67]:
x_gconv = torch.matmul(x_g, weights)  # b, N, dim_out

In [69]:
x_gconv.size()

torch.Size([24, 20])